In [1]:
#imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from hyperopt import hp
from hyperopt.pyll import scope
from catboost import CatBoostRegressor
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline, FeatureUnion

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from models.hyperopt_model import HyperoptModel
from models.utils import CustomDataFrameMapper
from models.utils import get_general_features_quora, _svr_features_transformations
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr,  spearmanr,  kendalltau

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
# read the data
preprocessed_data = pd.read_pickle('./data/quora.big_quantitative.pd')
# Create a dataframe
df = pd.DataFrame(preprocessed_data)
print(df.columns)

Index(['user_id', 'user_answers_num', 'user_questions_num', 'user_blogs_num',
       'user_posts_num', 'user_edits_num', 'user_followers_num',
       'user_followings_num', 'user_has_picture', 'user_topics_num',
       'user_bio', 'answer_id', 'answer_is_downvoted', 'answer_comments_num',
       'answer_content', 'answer_user_credential', 'score', 'answer_views_num',
       'answer_timestamp', 'answer_rank', 'question_id', 'question_title',
       'question_fetched_answers_num', 'question_expected_answers_num',
       'question_followers_num', 'question_comments_num',
       'user_fetched_answers_num', 'days_since_epoch', 'days_rescaled',
       'user_z_score', 'user_ff_ratio', 'max_answer_rank', 'answer_rank_ratio',
       'user_top_score_ratio', 'question_subjectivity', 'answer_smog_index',
       'answer_smog_min_age', 'user_obj_presense', 'user_subj_presense',
       'Unnamed: 0', 'lda_1', 'lda_2', 'lda_3', 'lda_4', 'lda_5', 'lda_6',
       'lda_7', 'lda_8', 'lda_9', 'lda_10', 'lda

In [3]:
# Create two new dataframes, one with the training rows, one with the test rows
train, test=train_test_split(df, test_size=0.2, random_state=0)

In [4]:
# Show the number of observations for the test and training dataframes
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 114411
Number of observations in the test data: 28603


In [5]:
#get features
answer_features_transformations, \
question_features_transformations, \
user_features_transformations = get_general_features_quora()

In [6]:
    #some parameters
    DO_LOWESS = False
    hyperopt = False
    # Run CatBoostRegressor with hyperopt optimization
    cat = HyperoptModel(train.copy(), test.copy(), 'cat', cv=3)
    cat.raw_features = []
    cat.pipeline = Pipeline([
        ('prepare_features', FeatureUnion([
            ('user_features', CustomDataFrameMapper(user_features_transformations)),
            ('time_features', CustomDataFrameMapper(time_features_transformations)),
            ('answer_features', CustomDataFrameMapper(answer_features_transformations)),
            ('question_features', CustomDataFrameMapper(question_features_transformations))
        ])),
        ('estimate', CatBoostRegressor(**{'bagging_temperature': 0.29793733267072053,
                                          'iterations': 5000,
                                          'l2_leaf_reg': 1.5511980979084095,
                                          'learning_rate': 0.08683045812519587,
                                          'loss_function': 'RMSE',
                                          'random_seed': 0,
                                          'random_strength': 20,
                                          'verbose': True,
                                          'train_dir': 'outputs/cat'}))
    ])
    
    for transformer in cat.pipeline.named_steps['prepare_features'].transformer_list:
        cat.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]
    
    cat.space = {
        'estimate__iterations': hp.choice('estimate__iterations', [50]),
        'estimate__loss_function': hp.choice('estimate__loss_function', ['RMSE']),
        'estimate__train_dir': hp.choice('estimate__train_dir', ['outputs/cat']),
        'estimate__thread_count': hp.choice('estimate__thread_count', [4]),
        'estimate__used_ram_limit': hp.choice('estimate__used_ram_limit', [1024 * 1024 * 1024 * 4]),  # 4gb
        'estimate__random_seed': hp.choice('estimate__random_seed', [0]),

        'estimate__learning_rate': hp.loguniform('estimate__learning_rate', -5, 0),
        'estimate__random_strength': hp.choice('estimate__random_strength', [1, 20]),
        'estimate__l2_leaf_reg': hp.loguniform('estimate__l2_leaf_reg', 0, np.log(10)),
        'estimate__bagging_temperature': hp.uniform('estimate__bagging_temperature', 0, 1),
    }

    """
    find the best number of trees (following https://tech.yandex.com/catboost/doc/dg/concepts/parameter-tuning_trees-number-docpage/#parameter-tuning_trees-number)
    need to also substitute the CatBoostRegressor parameters with:
        ('estimate', CatBoostRegressor(iterations=10000, loss_function='RMSE', auto_stop_pval=1e-4, use_best_model=True, train_dir='outputs/cat_trees', verbose=True))
    """
    # num_trees_train, num_trees_eval = train_test_split(train, test_size=0.2, random_state=0)
    # X = num_trees_train[list(filter(lambda column: column in cat.raw_features, cat.train.columns))]
    # y = num_trees_train['score']
    #
    # eval_X = num_trees_eval[list(filter(lambda column: column in cat.raw_features, cat.train.columns))]
    # eval_X = cat.pipeline.named_steps['prepare_features'].fit_transform(eval_X)
    # eval_set = (eval_X, num_trees_eval['score'])
    #
    # cat.pipeline.fit(X=X, y=y, estimate__eval_set=eval_set)
    # cat.model = cat.pipeline
    # print(cat.model.named_steps['estimate'].get_params())
    # cat.model.named_steps['estimate'].save_model('tmp/cat.model'.encode('utf-8'))

    if hyperopt:
        cat.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        cat.pipeline.fit(X=cat.X_train, y=cat.y_train)
        cat.model = cat.pipeline
        cat.stats()

        # joblib.dump(cat, './outputs/models/%s.pckl' % cat.output_prefix)
        # cat = joblib.load('./outputs/models/%s.pckl' % cat.output_prefix)

        cat.plot_predicted_vs_actual(do_lowess=False)
        cat.plot_residuals(r_type='raw', do_lowess=False)
        cat.plot_feature_importance()

Borders generated
0:	learn 0.5034394424passed: 0.229 sec	total: 1.11s	remaining: 1h 32m 20s
1:	learn 0.4617420782passed: 0.177 sec	total: 1.29s	remaining: 53m 33s
2:	learn 0.4238395137passed: 0.176 sec	total: 1.46s	remaining: 40m 35s
3:	learn 0.389409431passed: 0.189 sec	total: 1.65s	remaining: 34m 22s
4:	learn 0.3581937181passed: 0.176 sec	total: 1.83s	remaining: 30m 26s
5:	learn 0.3298876144passed: 0.176 sec	total: 2s	remaining: 27m 47s
6:	learn 0.3042402245passed: 0.179 sec	total: 2.18s	remaining: 25m 57s
7:	learn 0.2809361687passed: 0.178 sec	total: 2.36s	remaining: 24m 33s
8:	learn 0.2600170142passed: 0.183 sec	total: 2.54s	remaining: 23m 31s
9:	learn 0.2412321805passed: 0.18 sec	total: 2.72s	remaining: 22m 39s
10:	learn 0.224281573passed: 0.175 sec	total: 2.9s	remaining: 21m 55s
11:	learn 0.2090629892passed: 0.183 sec	total: 3.08s	remaining: 21m 21s
12:	learn 0.1950296232passed: 0.181 sec	total: 3.26s	remaining: 20m 52s
13:	learn 0.1823680924passed: 0.18 sec	total: 3.44s	remainin

115:	learn 0.0890890894passed: 0.183 sec	total: 22.6s	remaining: 15m 52s
116:	learn 0.08905564535passed: 0.179 sec	total: 22.8s	remaining: 15m 52s
117:	learn 0.08904444157passed: 0.182 sec	total: 23s	remaining: 15m 51s
118:	learn 0.08902965717passed: 0.195 sec	total: 23.2s	remaining: 15m 51s
119:	learn 0.08901177389passed: 0.208 sec	total: 23.4s	remaining: 15m 51s
120:	learn 0.08900196475passed: 0.193 sec	total: 23.6s	remaining: 15m 51s
121:	learn 0.08896856141passed: 0.207 sec	total: 23.8s	remaining: 15m 51s
122:	learn 0.08893914663passed: 0.199 sec	total: 24s	remaining: 15m 51s
123:	learn 0.08890025147passed: 0.187 sec	total: 24.2s	remaining: 15m 50s
124:	learn 0.0888822724passed: 0.188 sec	total: 24.4s	remaining: 15m 50s
125:	learn 0.08885806098passed: 0.225 sec	total: 24.6s	remaining: 15m 51s
126:	learn 0.08882952567passed: 0.187 sec	total: 24.8s	remaining: 15m 51s
127:	learn 0.08882010561passed: 0.208 sec	total: 25s	remaining: 15m 51s
128:	learn 0.08881165227passed: 0.198 sec	tota

227:	learn 0.08524464355passed: 0.219 sec	total: 45.1s	remaining: 15m 43s
228:	learn 0.08520889135passed: 0.218 sec	total: 45.3s	remaining: 15m 43s
229:	learn 0.08518643409passed: 0.22 sec	total: 45.5s	remaining: 15m 43s
230:	learn 0.08515667053passed: 0.207 sec	total: 45.7s	remaining: 15m 43s
231:	learn 0.08513701367passed: 0.193 sec	total: 45.9s	remaining: 15m 43s
232:	learn 0.08511426909passed: 0.194 sec	total: 46.1s	remaining: 15m 43s
233:	learn 0.08509685516passed: 0.186 sec	total: 46.3s	remaining: 15m 42s
234:	learn 0.08508141921passed: 0.198 sec	total: 46.5s	remaining: 15m 42s
235:	learn 0.08505535304passed: 0.213 sec	total: 46.7s	remaining: 15m 42s
236:	learn 0.08503564414passed: 0.191 sec	total: 46.9s	remaining: 15m 42s
237:	learn 0.08501321559passed: 0.193 sec	total: 47.1s	remaining: 15m 42s
238:	learn 0.08497339749passed: 0.202 sec	total: 47.3s	remaining: 15m 42s
239:	learn 0.08494656436passed: 0.186 sec	total: 47.5s	remaining: 15m 41s
240:	learn 0.08492068439passed: 0.194 s

339:	learn 0.08311609955passed: 0.253 sec	total: 1m 11s	remaining: 16m 20s
340:	learn 0.08310510473passed: 0.257 sec	total: 1m 11s	remaining: 16m 20s
341:	learn 0.08309759113passed: 0.244 sec	total: 1m 12s	remaining: 16m 20s
342:	learn 0.08309136551passed: 0.206 sec	total: 1m 12s	remaining: 16m 20s
343:	learn 0.08308146489passed: 0.226 sec	total: 1m 12s	remaining: 16m 20s
344:	learn 0.08307086643passed: 0.193 sec	total: 1m 12s	remaining: 16m 20s
345:	learn 0.08305977966passed: 0.223 sec	total: 1m 12s	remaining: 16m 20s
346:	learn 0.08304813669passed: 0.219 sec	total: 1m 13s	remaining: 16m 19s
347:	learn 0.08303669633passed: 0.193 sec	total: 1m 13s	remaining: 16m 19s
348:	learn 0.08301812136passed: 0.197 sec	total: 1m 13s	remaining: 16m 19s
349:	learn 0.08300261145passed: 0.196 sec	total: 1m 13s	remaining: 16m 18s
350:	learn 0.08298756863passed: 0.28 sec	total: 1m 13s	remaining: 16m 19s
351:	learn 0.08297637453passed: 0.449 sec	total: 1m 14s	remaining: 16m 22s
352:	learn 0.0829619943pas

449:	learn 0.08176510296passed: 0.192 sec	total: 1m 37s	remaining: 16m 29s
450:	learn 0.0817461747passed: 0.19 sec	total: 1m 38s	remaining: 16m 29s
451:	learn 0.08173691636passed: 0.197 sec	total: 1m 38s	remaining: 16m 28s
452:	learn 0.08172845575passed: 0.188 sec	total: 1m 38s	remaining: 16m 28s
453:	learn 0.08171652989passed: 0.225 sec	total: 1m 38s	remaining: 16m 27s
454:	learn 0.0817003569passed: 0.22 sec	total: 1m 38s	remaining: 16m 27s
455:	learn 0.08168693952passed: 0.197 sec	total: 1m 39s	remaining: 16m 27s
456:	learn 0.08167887652passed: 0.187 sec	total: 1m 39s	remaining: 16m 26s
457:	learn 0.08166771958passed: 0.185 sec	total: 1m 39s	remaining: 16m 26s
458:	learn 0.08165199422passed: 0.215 sec	total: 1m 39s	remaining: 16m 26s
459:	learn 0.08163935285passed: 0.231 sec	total: 1m 39s	remaining: 16m 26s
460:	learn 0.08162624411passed: 0.186 sec	total: 1m 40s	remaining: 16m 25s
461:	learn 0.08161786841passed: 0.235 sec	total: 1m 40s	remaining: 16m 25s
462:	learn 0.08160949429passe

...

4390:	learn 0.06547914204passed: 0.256 sec	total: 18m 8s	remaining: 2m 31s
4391:	learn 0.06547664808passed: 0.26 sec	total: 18m 9s	remaining: 2m 30s
4392:	learn 0.06547310523passed: 0.262 sec	total: 18m 9s	remaining: 2m 30s
4393:	learn 0.06546929189passed: 0.282 sec	total: 18m 9s	remaining: 2m 30s
4394:	learn 0.06546798888passed: 0.318 sec	total: 18m 9s	remaining: 2m 30s
4395:	learn 0.06546543991passed: 0.418 sec	total: 18m 10s	remaining: 2m 29s
4396:	learn 0.06546258879passed: 0.556 sec	total: 18m 10s	remaining: 2m 29s
4397:	learn 0.06546064684passed: 0.205 sec	total: 18m 11s	remaining: 2m 29s
4398:	learn 0.06545821203passed: 0.23 sec	total: 18m 11s	remaining: 2m 29s
4399:	learn 0.06545611564passed: 0.222 sec	total: 18m 11s	remaining: 2m 28s
4400:	learn 0.06545468338passed: 0.227 sec	total: 18m 11s	remaining: 2m 28s
4401:	learn 0.06545459198passed: 0.169 sec	total: 18m 11s	remaining: 2m 28s
4402:	learn 0.0654515018passed: 0.268 sec	total: 18m 12s	remaining: 2m 28s
4403:	learn 0.065447

4500:	learn 0.06520661657passed: 0.182 sec	total: 18m 34s	remaining: 2m 3s
4501:	learn 0.06520604478passed: 0.182 sec	total: 18m 35s	remaining: 2m 3s
4502:	learn 0.06520536286passed: 0.18 sec	total: 18m 35s	remaining: 2m 3s
4503:	learn 0.06520311722passed: 0.197 sec	total: 18m 35s	remaining: 2m 2s
4504:	learn 0.06519859965passed: 0.19 sec	total: 18m 35s	remaining: 2m 2s
4505:	learn 0.0651973033passed: 0.184 sec	total: 18m 35s	remaining: 2m 2s
4506:	learn 0.06519458164passed: 0.178 sec	total: 18m 35s	remaining: 2m 2s
4507:	learn 0.0651917537passed: 0.188 sec	total: 18m 36s	remaining: 2m 1s
4508:	learn 0.06518760239passed: 0.185 sec	total: 18m 36s	remaining: 2m 1s
4509:	learn 0.06518447884passed: 0.206 sec	total: 18m 36s	remaining: 2m 1s
4510:	learn 0.0651813642passed: 0.19 sec	total: 18m 36s	remaining: 2m 1s
4511:	learn 0.06517804217passed: 0.193 sec	total: 18m 36s	remaining: 2m
4512:	learn 0.06517417137passed: 0.189 sec	total: 18m 37s	remaining: 2m
4513:	learn 0.06517118014passed: 0.18

4609:	learn 0.06492626927passed: 0.185 sec	total: 18m 56s	remaining: 1m 36s
4610:	learn 0.0649242371passed: 0.189 sec	total: 18m 56s	remaining: 1m 35s
4611:	learn 0.06492076556passed: 0.195 sec	total: 18m 57s	remaining: 1m 35s
4612:	learn 0.06491896609passed: 0.186 sec	total: 18m 57s	remaining: 1m 35s
4613:	learn 0.06491634554passed: 0.19 sec	total: 18m 57s	remaining: 1m 35s
4614:	learn 0.06491432902passed: 0.2 sec	total: 18m 57s	remaining: 1m 34s
4615:	learn 0.06491199172passed: 0.183 sec	total: 18m 57s	remaining: 1m 34s
4616:	learn 0.06490962002passed: 0.189 sec	total: 18m 57s	remaining: 1m 34s
4617:	learn 0.06490540063passed: 0.197 sec	total: 18m 58s	remaining: 1m 34s
4618:	learn 0.06490229617passed: 0.204 sec	total: 18m 58s	remaining: 1m 33s
4619:	learn 0.06489817293passed: 0.196 sec	total: 18m 58s	remaining: 1m 33s
4620:	learn 0.06489685061passed: 0.19 sec	total: 18m 58s	remaining: 1m 33s
4621:	learn 0.06489565959passed: 0.194 sec	total: 18m 58s	remaining: 1m 33s
4622:	learn 0.064

4828:	learn 0.06441045891passed: 0.176 sec	total: 19m 37s	remaining: 41.7s
4829:	learn 0.06440838273passed: 0.183 sec	total: 19m 37s	remaining: 41.4s
4830:	learn 0.0644063965passed: 0.18 sec	total: 19m 37s	remaining: 41.2s
4831:	learn 0.0644047779passed: 0.181 sec	total: 19m 37s	remaining: 40.9s
4832:	learn 0.06440311233passed: 0.181 sec	total: 19m 37s	remaining: 40.7s
4833:	learn 0.06440002172passed: 0.179 sec	total: 19m 38s	remaining: 40.5s
4834:	learn 0.06439676803passed: 0.181 sec	total: 19m 38s	remaining: 40.2s
4835:	learn 0.06439471669passed: 0.179 sec	total: 19m 38s	remaining: 40s
4836:	learn 0.06439243215passed: 0.184 sec	total: 19m 38s	remaining: 39.7s
4837:	learn 0.06438934272passed: 0.185 sec	total: 19m 38s	remaining: 39.5s
4838:	learn 0.0643869155passed: 0.183 sec	total: 19m 39s	remaining: 39.2s
4839:	learn 0.06438508332passed: 0.181 sec	total: 19m 39s	remaining: 39s
4840:	learn 0.06438089092passed: 0.187 sec	total: 19m 39s	remaining: 38.7s
4841:	learn 0.06437856884passed: 

4938:	learn 0.06415300608passed: 0.186 sec	total: 19m 57s	remaining: 14.8s
4939:	learn 0.06415213178passed: 0.177 sec	total: 19m 57s	remaining: 14.5s
4940:	learn 0.06414934151passed: 0.182 sec	total: 19m 57s	remaining: 14.3s
4941:	learn 0.06414833579passed: 0.181 sec	total: 19m 57s	remaining: 14.1s
4942:	learn 0.06414522989passed: 0.185 sec	total: 19m 58s	remaining: 13.8s
4943:	learn 0.06414357946passed: 0.179 sec	total: 19m 58s	remaining: 13.6s
4944:	learn 0.06414235796passed: 0.18 sec	total: 19m 58s	remaining: 13.3s
4945:	learn 0.06414017724passed: 0.186 sec	total: 19m 58s	remaining: 13.1s
4946:	learn 0.06413608924passed: 0.183 sec	total: 19m 58s	remaining: 12.8s
4947:	learn 0.0641340903passed: 0.183 sec	total: 19m 58s	remaining: 12.6s
4948:	learn 0.06413062571passed: 0.187 sec	total: 19m 59s	remaining: 12.4s
4949:	learn 0.06412885614passed: 0.179 sec	total: 19m 59s	remaining: 12.1s
4950:	learn 0.06412725064passed: 0.181 sec	total: 19m 59s	remaining: 11.9s
4951:	learn 0.06412473161pa

In [6]:
    #some parameters
    DO_LOWESS = False
    hyperopt = False
    # Run LGBMRegressor with hyperopt optimization
    lgbm = HyperoptModel(train.copy(), test.copy(), 'lgbm', cv=5)
    lgbm.raw_features = []
    lgbm.pipeline = Pipeline([
        ('prepare_features', FeatureUnion([
            ('user_features', CustomDataFrameMapper(user_features_transformations)),
            #('time_features', CustomDataFrameMapper(time_features_transformations)),
            ('answer_features', CustomDataFrameMapper(answer_features_transformations)),
            ('question_features', CustomDataFrameMapper(question_features_transformations))
        ])),
        ('estimate', lgb.LGBMRegressor(**{'bagging_fraction': 0.9583593582453502,
                                          'feature_fraction': 0.797191970090108,
                                          'lambda_l1': 0,
                                          'lambda_l2': 0,
                                          'learning_rate': 0.06967397660277702,
                                          'min_data_in_leaf': 2,
                                          'min_sum_hessian_in_leaf': 3.8117576166032006,
                                          'n_estimators': 435,
                                          'num_leaves': 287,
                                          'objective': 'regression',
                                          'seed': 0}))
    ])
    for transformer in lgbm.pipeline.named_steps['prepare_features'].transformer_list:
        lgbm.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]

    """ find number of trees """
    # num_trees_train, num_trees_eval = train_test_split(train, test_size=0.2, random_state=0)
    # X = num_trees_train[list(filter(lambda column: column in lgbm.raw_features, lgbm.train.columns))]
    # y = num_trees_train['score']
    #
    # eval_X = num_trees_eval[list(filter(lambda column: column in lgbm.raw_features, lgbm.train.columns))]
    # eval_X = lgbm.pipeline.named_steps['prepare_features'].fit_transform(eval_X)
    # eval_set = (eval_X, num_trees_eval['score'])
    #
    # best = lgbm.pipeline.fit(X=X, y=y, estimate__eval_set=eval_set, estimate__early_stopping_rounds=10)
    # print(best.named_steps['estimate'].best_iteration)

    lgbm.space = {
        'estimate__objective': hp.choice('estimate__objective', ['regression']),
        'estimate__n_estimators': hp.choice('estimate__n_estimators', [400]),
        'estimate__seed': hp.choice('estimate__seed', [0]),

        'estimate__learning_rate': hp.loguniform('estimate__learning_rate', -7, 0),
        'estimate__num_leaves': scope.int(hp.qloguniform('estimate__num_leaves', 1, 7, 1)),
        'estimate__feature_fraction': hp.uniform('estimate__feature_fraction', 0.5, 1),
        'estimate__bagging_fraction': hp.uniform('estimate__bagging_fraction', 0.5, 1),
        'estimate__min_data_in_leaf': scope.int(hp.qloguniform('estimate__min_data_in_leaf', 0, 6, 1)),
        'estimate__min_sum_hessian_in_leaf': hp.loguniform('estimate__min_sum_hessian_in_leaf', -16, 5),
        'estimate__lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('estimate__lambda_l1_positive', -16, 2)]),
        'estimate__lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('estimate__lambda_l2_positive', -16, 2)]),
    }

    if hyperopt:
        lgbm.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        lgbm.pipeline.fit(X=lgbm.X_train, y=lgbm.y_train)
        lgbm.model = lgbm.pipeline
        lgbm.stats()
        lgbm.plot_predicted_vs_actual()
        lgbm.plot_residuals(r_type='raw', do_lowess=False)
        lgbm.plot_feature_importance()

Stats (train | test):
	R^2 score:		0.7285
					0.3895
	RMSE:			0.0556
					0.0834
	Mean error:		0.0429
					0.0639
	Pearson:		0.8709
					0.6241
	Spearman:		0.8618
					0.6313
	KendallTau:		0.6779
					0.4515

Plotting predicted vs. actual ...done

Plotting residuals ...done

Plotting feature importances ...done



In [8]:
    #some parameters
    DO_LOWESS = False
    hyperopt = False
    # Run RandomForestRegressor with hyperopt optimization
    rf = HyperoptModel(train.copy(), test.copy(),'rf', cv=3, max_evals = 10)
    rf.raw_features = []
    rf.pipeline = Pipeline([
        ('prepare_features', FeatureUnion([
            ('user_features', CustomDataFrameMapper(user_features_transformations)),
            ('time_features', CustomDataFrameMapper(time_features_transformations)),
            ('answer_features', CustomDataFrameMapper(answer_features_transformations)),
            ('question_features', CustomDataFrameMapper(question_features_transformations))
        ])),
        ('estimate', RandomForestRegressor(**{'max_features': 0.5907165396346349,
                                              'min_samples_leaf': 10,
                                              'n_estimators': 2208,
                                              'oob_score': True,
                                              'random_state': 0}))
    ])
    for transformer in rf.pipeline.named_steps['prepare_features'].transformer_list:
        rf.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]

    rf.space = {
        'estimate__random_state': hp.choice('estimate__random_state', [0]),
        'estimate__oob_score': hp.choice('estimate__oob_score', [True]),

        'estimate__max_features': hp.uniform('estimate__max_features', 0, 1.),
        'estimate__n_estimators': hp.choice('estimate__n_estimators', range(1, 3000 + 1)),
        #'estimate__criterion': hp.choice('estimate__criterion', ['gini', 'entropy']),
        'estimate__min_samples_leaf': hp.choice('estimate__min_samples_leaf', range(1, 100 + 1)),
        #'estimate__scale': hp.choice('estimate__scale', [0, 1.]),
        #'estimate__normalize': hp.choice('estimate__normalize', [0, 1.]),
    }

    if hyperopt:
        rf.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        rf.pipeline.fit(X=rf.X_train, y=rf.y_train)
        rf.model = rf.pipeline
        rf.stats()
        rf.plot_predicted_vs_actual()
        rf.plot_residuals(r_type='raw', do_lowess=False)
        rf.plot_feature_importance()

Stats (train | test):
	R^2 score:		0.6091
					0.3897
	RMSE:			0.0667
					0.0834
	Mean error:		0.0502
					0.0634
	Pearson:		0.7955
					0.6243
	Spearman:		0.8069
					0.6355
	KendallTau:		0.6151
					0.4557

Plotting predicted vs. actual ...done

Plotting residuals ...done

Plotting feature importances ...done



In [9]:
    #some parameters
    DO_LOWESS = False
    hyperopt = False
    # Run SVR with hyperopt optimization
    answer_features_transformations, question_features_transformations, time_features_transformations, user_features_transformations = _svr_features_transformations()

    svm = HyperoptModel(train.copy(), test.copy(), 'svr', cv=3)
    svm.pipeline = Pipeline([
        ('prepare_features', FeatureUnion([
            ('user_features', CustomDataFrameMapper(user_features_transformations)),
            ('time_features', CustomDataFrameMapper(time_features_transformations)),
            ('answer_features', CustomDataFrameMapper(answer_features_transformations)),
            ('question_features', CustomDataFrameMapper(question_features_transformations))
        ])),
        ('estimate', SVR(C=3.376124349816575, gamma=0.0069678844996990535, kernel='rbf'))
    ])
    svm.raw_features = []
    for transformer in svm.pipeline.named_steps['prepare_features'].transformer_list:
        svm.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]

    """
    1) The C parameter trades off misclassification of training examples against simplicity of the decision surface. 
    A low C makes the decision surface smooth, while a high C aims at classifying all training examples correctly 
        by giving the model freedom to select more samples as support vectors.
        
    2)  The gamma parameter defines how far the influence of a single training example reaches, 
    with low values meaning ‘far’ and high values meaning ‘close’. 
    The gamma parameters can be seen as the inverse of the 
        radius of influence of samples selected by the model as support vectors.
    """

    # default_gamma = 1. / len(svm.raw_features)
    svm.space = {
        'estimate__C': hp.uniform('estimate__C', 0, 10.),
        'estimate__kernel': hp.choice('estimate__kernel', ['linear', 'sigmoid', 'rbf']),
        'estimate__gamma': hp.uniform('estimate__gamma', 0, 10.),
    }

    if hyperopt:
        svm.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        svm.pipeline.fit(X=svm.X_train, y=svm.y_train)
        svm.model = svm.pipeline
        svm.stats()
        svm.plot_predicted_vs_actual()
        svm.plot_residuals(r_type='raw', do_lowess=False)
        svm.plot_feature_importance()

Stats (train | test):
	R^2 score:		0.4494
					0.3180
	RMSE:			0.0791
					0.0881
	Mean error:		0.0637
					0.0686
	Pearson:		0.6777
					0.5688
	Spearman:		0.6773
					0.5801
	KendallTau:		0.4885
					0.4096

Plotting predicted vs. actual ...done

Plotting residuals ...done



In [7]:
    #some parameters
    DO_LOWESS = False
    hyperopt = True
    # Run MLPRegressor with hyperopt optimization
    nn = HyperoptModel(train.copy(), test.copy(),'nn', cv=3, max_evals=100)
    nn.raw_features = []

    nn.pipeline = Pipeline([
         ('prepare_features', FeatureUnion([
            ('user_features', CustomDataFrameMapper(user_features_transformations)),
            ('time_features', CustomDataFrameMapper(time_features_transformations)),
            ('answer_features', CustomDataFrameMapper(answer_features_transformations)),
            ('question_features', CustomDataFrameMapper(question_features_transformations))
        ])),
        ('estimate', MLPRegressor())
    ])
    
    for transformer in nn.pipeline.named_steps['prepare_features'].transformer_list:
        nn.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]
    


    nn.space = {
         'estimate__alpha' : hp.uniform('estimate__alpha', 0.00001, 1),
         'estimate__activation' : hp.choice('estimate__activation', ['logistic']), # 'identity', 'logistic', 'tanh', 'relu'
         #'estimate__learning_rate' : hp.choice('estimate__learning_rate', ['constant', 'invscaling', 'adaptive']),
         'estimate__hidden_layer_sizes' : (scope.int(hp.uniform('estimate__first_layer', 1, 100))),
         'estimate__solver' : hp.choice('estimate__solver', ['adam']), #'lbfgs', 'sgd',
         #'estimate__max_iter' : scope.int(hp.uniform('estimate__max_iter', 500, 1000))
    }

    if hyperopt:
        nn.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        nn.pipeline.fit(X=nn.X_train, y=nn.y_train)
        nn.model = nn.pipeline
        nn.stats()
        nn.plot_predicted_vs_actual(do_lowess=DO_LOWESS)
        nn.plot_feature_importance()
        nn.qq_plot()

Performing parameters optimization...
[1/100]	cv_eval_time=335.20 sec	RMSE=0.106372	R^2=-0.002218
[2/100]	cv_eval_time=345.03 sec	RMSE=0.106391	R^2=-0.011225
[3/100]	cv_eval_time=361.85 sec	RMSE=0.106283	R^2=-0.008827
[4/100]	cv_eval_time=352.93 sec	RMSE=0.105946	R^2=-0.012185
[5/100]	cv_eval_time=341.24 sec	RMSE=0.106091	R^2=-0.000501
[6/100]	cv_eval_time=344.44 sec	RMSE=0.108364	R^2=0.010731
[7/100]	cv_eval_time=315.00 sec	RMSE=0.106169	R^2=-0.000413
[8/100]	cv_eval_time=329.25 sec	RMSE=0.105362	R^2=0.009370
[9/100]	cv_eval_time=337.77 sec	RMSE=0.106850	R^2=-0.013371
[10/100]	cv_eval_time=321.44 sec	RMSE=0.106537	R^2=0.010055
[11/100]	cv_eval_time=330.16 sec	RMSE=0.105916	R^2=-0.008729
[12/100]	cv_eval_time=332.60 sec	RMSE=0.106702	R^2=-0.006775
[13/100]	cv_eval_time=336.99 sec	RMSE=0.106635	R^2=0.016113
[14/100]	cv_eval_time=329.66 sec	RMSE=0.106594	R^2=0.006906
[15/100]	cv_eval_time=324.41 sec	RMSE=0.107505	R^2=-0.013101
[16/100]	cv_eval_time=339.50 sec	RMSE=0.106847	R^2=0.023502
[

In [ ]:
    #remove string and datetime columns
    dfPoly = df.copy()
    dfPoly = dfPoly.select_dtypes(exclude=['object','datetime64'])
    y = dfPoly['score'].copy()
    dfPoly.drop('score', axis=1, inplace=True) 
    dfPoly.drop('user_id', axis=1, inplace=True) 
    #get rid of subjectivity and answer columns
    dfPoly = dfPoly[['user_answers_num', 'user_questions_num', 'user_blogs_num',
       'user_posts_num', 'user_edits_num', 'user_followers_num',
       'user_followings_num', 'user_has_picture', 'user_topics_num',
       'answer_id', 'answer_is_downvoted', 'answer_comments_num',
       'answer_upvotes_num', 'answer_views_num', 'answer_rank',
       'question_fetched_answers_num', 'question_expected_answers_num',
       'question_followers_num', 'question_comments_num',
       'user_fetched_answers_num', 'days_since_epoch', 'days_rescaled',
       'user_z_score', 'user_ff_ratio', 'max_answer_rank', 'answer_rank_ratio',
       'user_top_score_ratio']]
    #Polynomial linear regression 
    degrees = 4
    for d in range(1, degrees+1):
            print("Degree: %s" % d)
            # Create the model, split the sets and fit it
            polynomial_features = PolynomialFeatures(
                degree=d, include_bias=False
            )
            linear_regression = linear_model.LinearRegression()
            model = Pipeline([
                ("polynomial_features", polynomial_features),
                ("linear_regression", linear_regression)
            ])
           
            # create training and testing vars
            X_train, X_test, y_train, y_test = train_test_split(dfPoly, y, test_size=0.2)

            model.fit(X_train, y_train)
            # Calculate the metrics
            train_prediction = model.predict(X_train)
            test_prediction = model.predict(X_test)
            
            print('Polynomial degree: {}'.format(d))
            print('Stats (train | test):')
            print('\tR^2 score:\t\t%.4f\n\t\t\t\t\t%.4f' % (r2_score(y_train, train_prediction),
                                                        r2_score(y_test, test_prediction)))
            print('\tRMSE:\t\t\t%.4f\n\t\t\t\t\t%.4f' % (mean_squared_error(y_train, train_prediction) ** 0.5,
                                                     mean_squared_error(y_test, test_prediction) ** 0.5))
            print('\tMean error:\t\t%.4f\n\t\t\t\t\t%.4f' % (mean_absolute_error(y_train, train_prediction),
                                                         mean_absolute_error(y_test, test_prediction)))
            print('\tPearson:\t\t%.4f\n\t\t\t\t\t%.4f' % (pearsonr(y_train, train_prediction)[0],
                                                         pearsonr(y_test, test_prediction)[0]))
            print('\tSpearman:\t\t%.4f\n\t\t\t\t\t%.4f' % (spearmanr(y_train, train_prediction)[0],
                                                         spearmanr(y_test, test_prediction)[0]))
            print('\tKendallTau:\t\t%.4f\n\t\t\t\t\t%.4f' % (kendalltau(y_train, train_prediction)[0],
                                                         kendalltau(y_test, test_prediction)[0]))
            

Degree: 1
Polynomial degree: 1
Stats (train | test):
	R^2 score:		0.1881
					0.2100
	RMSE:			0.0962
					0.0944
	Mean error:		0.0743
					0.0744
	Pearson:		0.4337
					0.4616
	Spearman:		0.4971
					0.4923
	KendallTau:		0.3442
					0.3406
Degree: 2


Index(['user_answers_num', 'user_questions_num', 'user_blogs_num',
       'user_posts_num', 'user_edits_num', 'user_followers_num',
       'user_followings_num', 'user_has_picture', 'user_topics_num',
       'answer_id', 'answer_is_downvoted', 'answer_comments_num',
       'answer_upvotes_num', 'answer_views_num', 'answer_rank', 'question_id',
       'question_fetched_answers_num', 'question_expected_answers_num',
       'question_followers_num', 'question_comments_num',
       'user_fetched_answers_num', 'days_since_epoch', 'days_rescaled',
       'user_z_score', 'user_ff_ratio', 'max_answer_rank', 'answer_rank_ratio',
       'user_top_score_ratio', 'question_subjectivity', 'answer_smog_index',
       'answer_smog_min_age', 'user_obj_presense', 'user_subj_presense',
       'Unnamed: 0', 'lda_1', 'lda_2', 'lda_3', 'lda_4', 'lda_5', 'lda_6',
       'lda_7', 'lda_8', 'lda_9', 'lda_10', 'lda_11', 'lda_12', 'lda_13',
       'lda_14', 'lda_15', 'lda_16', 'lda_17', 'lda_18', 'lda_19', 'lda_2